In [1]:
import msoffcrypto
import io
from pyxlsb import open_workbook
import pandas as pd

file_path = '/Users/queencyangela/Desktop/M AMF/Second Sem/MATH 298.1/Metrohome offset - December.xlsb'
password = "CRQAD_amf_2025!"

decrypted = io.BytesIO()
with open(file_path, "rb") as f:
    office_file = msoffcrypto.OfficeFile(f)
    office_file.load_key(password=password)
    office_file.decrypt(decrypted)

# Now try opening the decrypted file
decrypted.seek(0)  # Reset file pointer

with open_workbook(decrypted) as wb:
    sheet_name = list(wb.sheets)[0]  # Get the first sheet name
    print(f"Opening Sheet: {sheet_name}")

    data = []
    with wb.get_sheet(sheet_name) as sheet:
        for row in sheet.rows():
            data.append([item.v for item in row])

# Step 3: Convert to Pandas DataFrame
df = pd.DataFrame(data)  
df.columns = df.iloc[0]  # Set first row as column names
df = df[1:].reset_index(drop=True)  # Remove the first row and reset index

# Show the DataFrame
print(df)

Opening Sheet: All Data
0         ILS_ACCT_NUM ORIG_BOOK_DATE    MATURITY CONSOLIDATION 122024  \
0      261989000821890     08/16/2006  08/16/2011                  1.0   
1      261989000679341     03/24/2010  03/24/2013                  2.0   
2      266428000283867     07/12/2002  07/12/2012                  7.0   
3      261614000688824     01/11/2010  01/11/2013                  9.0   
4      266428000217614     01/26/2001  02/28/2015                 13.0   
...                ...            ...         ...                  ...   
54995  232354008849721     12/27/2024  12/27/2034    181859003392614.0   
54996  232428008863268     12/05/2024  08/15/2026              80462.0   
54997  232428008863240     12/05/2024  08/15/2026              80463.0   
54998  232428008863909     12/05/2024  08/15/2026              80464.0   
54999  232470008840318     12/27/2024  12/27/2031    181960003390843.0   

0     MONTH BOOKED YEAR BOOKED 01/31/2010 02/28/2010 03/31/2010 04/30/2010  \
0        

In [2]:
# Convert "Maturity" column to datetime format
df["MATURITY"] = pd.to_datetime(df["MATURITY"], format="%m/%d/%Y")

# Define the cutoff date and filter data
cutoff_date = pd.Timestamp("2017-01-01")
df_filtered = df[df["MATURITY"] >= cutoff_date]

# Convert relevant columns to datetime
df_filtered["ORIG_BOOK_DATE"] = pd.to_datetime(df_filtered["ORIG_BOOK_DATE"], format="%m/%d/%Y")

obs_start = pd.Timestamp("2015-01-01")

# Retain only the first 3 columns and drop columns 4-6
df_filtered = df_filtered.iloc[:, [0, 1, 2] + list(range(6, df_filtered.shape[1]))]

# Identify and convert date columns
date_columns = df_filtered.columns[3:]  # Columns from the 7th position onward
date_columns_dt = pd.to_datetime(date_columns, errors="coerce")  # Convert to datetime

# Keep only date columns from Jan 2015 onwards
filtered_date_columns = date_columns[date_columns_dt >= obs_start]

# Retain only the selected columns
df_filtered = df_filtered.iloc[:, :3].join(df_filtered[filtered_date_columns])

# Print the cleaned DataFrame
print(df_filtered)

/var/folders/9v/t53h__5j3z9844qlc2pjpytc0000gn/T/ipykernel_825/3626142717.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["ORIG_BOOK_DATE"] = pd.to_datetime(df_filtered["ORIG_BOOK_DATE"], format="%m/%d/%Y")


0         ILS_ACCT_NUM ORIG_BOOK_DATE   MATURITY 01/31/2015 02/28/2015  \
10     261883000301915     2010-06-24 2020-01-26     CLOSED     CLOSED   
11     261883000395170     2010-08-05 2020-08-05     CLOSED     CLOSED   
12     232053000580470     2008-07-07 2018-03-25     CLOSED     CLOSED   
13     261041000699563     2009-12-14 2022-12-17        0.0        0.0   
27     261787000119585     2007-10-31 2021-11-07     CLOSED     CLOSED   
...                ...            ...        ...        ...        ...   
54995  232354008849721     2024-12-27 2034-12-27       None       None   
54996  232428008863268     2024-12-05 2026-08-15       None       None   
54997  232428008863240     2024-12-05 2026-08-15       None       None   
54998  232428008863909     2024-12-05 2026-08-15       None       None   
54999  232470008840318     2024-12-27 2031-12-27       None       None   

0     03/31/2015 04/30/2015 05/31/2015 06/30/2015 07/31/2015  ... 03/31/2024  \
10        CLOSED     CLOSED    

In [3]:
def expand_row_for_month_range(row, date_columns, date_columns_dt, start_month, end_month):
    book_date = row["ORIG_BOOK_DATE"]
    
    # Determine the starting index for M1
    if book_date < pd.Timestamp("2015-01-01"):
        start_idx = (date_columns_dt == pd.Timestamp("2015-01-31")).argmax()
    else:
        start_idx = (date_columns_dt > book_date).argmax()

    # Shift start index by (start_month - 1) to start at the desired month
    start_idx += (start_month - 1)

    # Calculate number of months to fetch
    num_months = end_month - start_month + 1

    # Get columns for the specified range
    valid_cols = date_columns[start_idx:start_idx + num_months]

    rows = []
    for i, col_name in enumerate(valid_cols, start=start_month):
        value = row[col_name]
        rows.append([
            row["ILS_ACCT_NUM"],               # assuming column 0 is ID
            row["ORIG_BOOK_DATE"],             # assuming column 1 is book date
            row["MATURITY"],                  # replace with actual 3rd column if needed
            f"M{i}",                           # Month label based on start_month
            col_name,                          # Actual date column name
            value
        ])

    return rows

In [14]:
all_expanded_rows = []
for _, row in df_filtered.iterrows():
    all_expanded_rows.extend(expand_row_for_month_range(row, date_columns, date_columns_dt, start_month=97, end_month=120))

df_long = pd.DataFrame(all_expanded_rows, columns=["ILS_ACCT_NUM", "ORIG_BOOK_DATE", "MATURITY", "MONTH", "DATE_COLUMN", "DELQ_AGE_ACTUAL"])

print(df_long)

           ILS_ACCT_NUM ORIG_BOOK_DATE   MATURITY MONTH DATE_COLUMN  \
0       261883000301915     2010-06-24 2020-01-26   M97  01/31/2023   
1       261883000301915     2010-06-24 2020-01-26   M98  02/28/2023   
2       261883000301915     2010-06-24 2020-01-26   M99  03/31/2023   
3       261883000301915     2010-06-24 2020-01-26  M100  04/30/2023   
4       261883000301915     2010-06-24 2020-01-26  M101  05/31/2023   
...                 ...            ...        ...   ...         ...   
515970  239651000983757     2005-09-14 2020-05-20  M116  08/31/2024   
515971  239651000983757     2005-09-14 2020-05-20  M117  09/30/2024   
515972  239651000983757     2005-09-14 2020-05-20  M118  10/31/2024   
515973  239651000983757     2005-09-14 2020-05-20  M119  11/30/2024   
515974  239651000983757     2005-09-14 2020-05-20  M120  12/31/2024   

       DELQ_AGE_ACTUAL  
0               CLOSED  
1               CLOSED  
2               CLOSED  
3               CLOSED  
4               CLOSED

In [15]:
# List of keywords to check in DELQ_AGE_ACTUAL
remove_accs = ['CLOSED', 'WRITE-OFF', 'PAID-OFF', 'ROPOA']

# Find IDs that contain any of the exclusion keywords in DELQ_AGE_ACTUAL
ids_to_exclude = df_long[df_long['DELQ_AGE_ACTUAL'].isin(remove_accs)]['ILS_ACCT_NUM'].unique()

# Filter out all rows with those IDs
df_long_filtered = df_long[~df_long['ILS_ACCT_NUM'].isin(ids_to_exclude)].copy()

print(df_long_filtered)

           ILS_ACCT_NUM ORIG_BOOK_DATE   MATURITY MONTH DATE_COLUMN  \
288     261050000354425     2010-10-14 2025-10-25   M97  01/31/2023   
289     261050000354425     2010-10-14 2025-10-25   M98  02/28/2023   
290     261050000354425     2010-10-14 2025-10-25   M99  03/31/2023   
291     261050000354425     2010-10-14 2025-10-25  M100  04/30/2023   
292     261050000354425     2010-10-14 2025-10-25  M101  05/31/2023   
...                 ...            ...        ...   ...         ...   
515850  232428002559483     2014-05-16 2039-05-16  M116  08/31/2024   
515851  232428002559483     2014-05-16 2039-05-16  M117  09/30/2024   
515852  232428002559483     2014-05-16 2039-05-16  M118  10/31/2024   
515853  232428002559483     2014-05-16 2039-05-16  M119  11/30/2024   
515854  232428002559483     2014-05-16 2039-05-16  M120  12/31/2024   

       DELQ_AGE_ACTUAL  
288                0.0  
289                4.0  
290                0.0  
291                0.0  
292                7.0

In [16]:
# Count the number of rows for each ID
id_counts = df_long_filtered['ILS_ACCT_NUM'].value_counts()

# Keep only IDs that appear exactly 24 times
valid_ids = id_counts[id_counts == 24].index

# Filter the DataFrame to keep only those valid IDs
df_cleaned = df_long_filtered[df_long_filtered['ILS_ACCT_NUM'].isin(valid_ids)].copy()

print(df_cleaned)

           ILS_ACCT_NUM ORIG_BOOK_DATE   MATURITY MONTH DATE_COLUMN  \
288     261050000354425     2010-10-14 2025-10-25   M97  01/31/2023   
289     261050000354425     2010-10-14 2025-10-25   M98  02/28/2023   
290     261050000354425     2010-10-14 2025-10-25   M99  03/31/2023   
291     261050000354425     2010-10-14 2025-10-25  M100  04/30/2023   
292     261050000354425     2010-10-14 2025-10-25  M101  05/31/2023   
...                 ...            ...        ...   ...         ...   
515850  232428002559483     2014-05-16 2039-05-16  M116  08/31/2024   
515851  232428002559483     2014-05-16 2039-05-16  M117  09/30/2024   
515852  232428002559483     2014-05-16 2039-05-16  M118  10/31/2024   
515853  232428002559483     2014-05-16 2039-05-16  M119  11/30/2024   
515854  232428002559483     2014-05-16 2039-05-16  M120  12/31/2024   

       DELQ_AGE_ACTUAL  
288                0.0  
289                4.0  
290                0.0  
291                0.0  
292                7.0

In [17]:
# Identify accounts that have 'NPL' in DELQ_AGE_ACTUAL for MONTH M1 to M12
accounts_to_remove = df_cleaned[
    (df_cleaned['MONTH'].isin([f'M{i}' for i in range(97, 108)])) &
    (df_cleaned['DELQ_AGE_ACTUAL'] == 'NPL')
]['ILS_ACCT_NUM'].unique()

# Filter out rows where ILS_ACCT_NUM is in the identified accounts
df_final = df_cleaned[~df_cleaned['ILS_ACCT_NUM'].isin(accounts_to_remove)]

print(df_final)

           ILS_ACCT_NUM ORIG_BOOK_DATE   MATURITY MONTH DATE_COLUMN  \
288     261050000354425     2010-10-14 2025-10-25   M97  01/31/2023   
289     261050000354425     2010-10-14 2025-10-25   M98  02/28/2023   
290     261050000354425     2010-10-14 2025-10-25   M99  03/31/2023   
291     261050000354425     2010-10-14 2025-10-25  M100  04/30/2023   
292     261050000354425     2010-10-14 2025-10-25  M101  05/31/2023   
...                 ...            ...        ...   ...         ...   
515850  232428002559483     2014-05-16 2039-05-16  M116  08/31/2024   
515851  232428002559483     2014-05-16 2039-05-16  M117  09/30/2024   
515852  232428002559483     2014-05-16 2039-05-16  M118  10/31/2024   
515853  232428002559483     2014-05-16 2039-05-16  M119  11/30/2024   
515854  232428002559483     2014-05-16 2039-05-16  M120  12/31/2024   

       DELQ_AGE_ACTUAL  
288                0.0  
289                4.0  
290                0.0  
291                0.0  
292                7.0

In [18]:
# Save DataFrame to CSV (no encryption, no index column)
df_final.to_csv("Metrohome offset filtered M97-120.csv", index=False)